In [31]:
import bs4
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

프로젝트에 필요한 라이브러리들을 임포트합니다. Beautiful Soup(bs4)를 웹 스크래핑에, LangChain 관련 라이브러리들을 LLM 연동 및 문서 처리에, OpenAI API를 사용하기 위한 모듈을 불러옵니다.
# LLM 설정 및 API 키 확인

In [32]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"API key exists: {api_key is not None}")

API key exists: True


OpenAI의 gpt-4o-mini 모델을 사용하기 위한 초기 설정을 합니다. .env 파일에서 환경변수를 불러와 API 키를 확인하고, 키가 존재하는지 확인합니다.

# 웹페이지 불러오기

In [33]:
from langchain_community.document_loaders import UnstructuredURLLoader

file_path = ["https://spartacodingclub.kr/blog/all-in-challenge_winner"]

loader = UnstructuredURLLoader(file_path)

docs = loader.load()
print(f"로드된 문서 수: {len(docs)}")
print(f"첫 번째 문서 내용 일부: {docs[0].page_content[:100] if docs else 'No content'}")


로드된 문서 수: 1
첫 번째 문서 내용 일부: 

포인트

로딩중

쿠폰

내 강의실

국비 신청 내역

수강권

증명서

숙제 피드백

계정

로그아웃

1725353737651-%C3%A1%C2%84%C2%8F%C3%A1%


LangChain의 WebBaseLoader를 사용하여 스파르타코딩클럽의 ALL-in 코딩 공모전 수상작 페이지를 불러옵니다.
Beautiful Soup의 SoupStrainer를 활용해 문서의 내용이 들어있는 ("css-j3idia", "editedContent")부분을 파싱하여 필요한 콘텐츠만 추출합니다.
로드된 문서의 수와 첫 문서의 일부분을 출력하여 정상적으로 데이터가 불러와졌는지 확인합니다.

# 문서 청크로 나누기



In [34]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10
)

splits = text_splitter.split_documents(docs)
print(f"분할된 청크 수: {len(splits)}")

if len(splits) == 0:
    print("청크가 없어 원본 문서를 사용합니다.")
    splits = docs

분할된 청크 수: 18


불러온 문서를 처리하기 쉽도록 청크(chunks)로 나눕니다. RecursiveCharacterTextSplitter를 사용하여 문서를 500자 단위로 분할하고, 청크 간 10자의 중복을 허용합니다. 분할된 청크의 수를 출력하고, 만약 청크가 생성되지 않았다면 원본 문서를 그대로 사용합니다.

# 벡터 스토어 생성 및 리트리버 설정

In [57]:
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
)
vectorstore_chroma = Chroma.from_documents( # self query retriever는 faiss와 호환되지 않음음
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
)
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 50}
)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


분할된 문서들을 벡터화하고, Chroma 벡터 데이터베이스에 저장합니다.
이후 이 벡터 스토어에서 문서를 검색할 수 있는 리트리버(retriever)를 생성합니다.
기본 검색 방식은 "similarity"(유사도 기반)이며, 최대 50개의 관련 문서를 반환하도록 설정합니다.

# 사용자 질문 및 문서 검색

In [36]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

print(len(retrieved_docs))
retrieved_docs

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


18


[Document(id='20062b27-219d-4327-a130-4225d427cd8f', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='조회수 752·6분 분량\n\n2024. 9. 3.\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'),
 Document(id='9d2916a9-1d4b-4ea3-b3f2-5ae73dd99f30', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content="포인트\n\n로딩중\n\n쿠폰\n\n내 강의실\n\n국비 신청 내역\n\n수강권\n\n증명서\n\n숙제 피드백\n\n계정\n\n로그아웃\n\n1725353737651-%C3%A1%C2%84%C2%8F%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%83%C3%A1%C2%85%C2%B5%C3%A1%C2%86%C2%BC%C3%A1%C2%84%C2%80%C3%A1%C2%85%C2%A9%C3%A1%C2%86%C2%BC%C3%A1%C2%84%C2%86%C3%A1%C2%85%C2%A9%C3%A1%C2%84%C2%8C%C

사용자 질문을 정의하고, 해당 질문에 관련된 문서를 리트리버를 통해 검색합니다. 검색된 문서의 수를 출력하고, 검색된 문서 목록을 확인합니다.

# 프롬프트 생성

In [48]:
prompt = ChatPromptTemplate.from_template("""
<Context>
당신은 공모전 심사위원으로, 검색된 문서 정보를 기반으로 질문에 답변하는 역할입니다.
검색된 문서: {context}
질문: {question}
</Context>

<Objective>
공모전에 대한 유익한 정보를 제공하고, 향후 참가자들이 참고할 수 있는 정보를 정리하는 것입니다.
</Objective>

<Style>
- 검색된 문서 내용을 바탕으로 사실적이고 객관적인 정보만 제공합니다.
- 개인적인 의견이나 해석 없이 검색 결과에서 찾은 정보만 활용합니다.
- 정보를 체계적으로 구조화하여 보기 쉽게 정리합니다.
- 검색 결과에서 찾을 수 없는 정보에 대해서는 명확히 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답변합니다.
</Style>

<Tone>
전문적이고 객관적인 어조로 답변합니다.
</Tone>

<Audience>
향후 이 공모전에 참가할 예정인 잠재적 참가자들입니다.
</Audience>

<Response>
제공된 문서 정보를 체계적으로 분석하여 질문에 대한 정확한 답변을 한국어로 제공하세요. 답변은:
1. 주요 정보를 카테고리별로 명확하게 구분하여 제시
2. 필요시 제목, 소제목, 글머리 기호 등을 활용해 가독성 향상
3. 검색 결과에서 찾은 모든 관련 정보를 포함
4. 답변을 찾을 수 없는 경우 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 명시
</Response>""")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n<Context>\n당신은 공모전 심사위원으로, 검색된 문서 정보를 기반으로 질문에 답변하는 역할입니다.\n검색된 문서: {context}\n질문: {question}\n</Context>\n\n<Objective>\n공모전에 대한 유익한 정보를 제공하고, 향후 참가자들이 참고할 수 있는 정보를 정리하는 것입니다.\n</Objective>\n\n<Style>\n- 검색된 문서 내용을 바탕으로 사실적이고 객관적인 정보만 제공합니다.\n- 개인적인 의견이나 해석 없이 검색 결과에서 찾은 정보만 활용합니다.\n- 정보를 체계적으로 구조화하여 보기 쉽게 정리합니다.\n- 검색 결과에서 찾을 수 없는 정보에 대해서는 명확히 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답변합니다.\n</Style>\n\n<Tone>\n전문적이고 객관적인 어조로 답변합니다.\n</Tone>\n\n<Audience>\n향후 이 공모전에 참가할 예정인 잠재적 참가자들입니다.\n</Audience>\n\n<Response>\n제공된 문서 정보를 체계적으로 분석하여 질문에 대한 정확한 답변을 한국어로 제공하세요. 답변은:\n1. 주요 정보를 카테고리별로 명확하게 구분하여 제시\n2. 필요시 제목, 소제목, 글머리 기호 등을 활용해 가독성 향상\n3. 검색 결과에서 찾은 모든 관련 정보를 포함\n4. 답변을 찾을 수 없는 경우 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 명시\n</Response>'), additional_kwargs={})]


LLM에게 전달할 프롬프트 템플릿을 생성합니다. 프롬프트는 공모전 심사위원 역할을 부여하고, 검색된 문서의 정보만을 활용하여 객관적인 정보를 제공하도록 지시합니다.

# LLM 응답 요청
검색된 문서와 사용자 질문을 프롬프트에 넣고, LLM에 응답을 요청한 후 결과를 출력합니다.

In [49]:
response = llm.invoke(user_prompt)
print(response.content)

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### All-in 코딩 공모전 수상작 요약

**공모전 개요**
- **목적**: 대학생들이 캠퍼스에서 겪는 불편함과 문제를 창의적인 아이디어로 해결하기 위한 대회
- **참가형태**: 팀 프로젝트 기반으로 다양한 웹/앱 서비스 개발

---

**대상 수상작**
1. **Lexi Note**
   - **서비스 설명**: 언어 공부를 위한 필기 웹 서비스. 단어 드래그 시 사전 연동, 번역 기능, 할 일 목록 및 스케줄 제공.
   - **기술 스택**: FE - Spring Boot, MySQL, Spring WebSocket / BE - React Native, TanStack Query, Axios

---

**우수상 수상작**
1. **우리집 히어로즈**
   - **서비스 설명**: 벌레 퇴치 히어로 매칭 서비스. 자취생이 벌레 문제를 겪을 때, 벌레 퇴치 진행자를 매칭해주는 서비스.
   - **기술 스택**: FE - React Native, Expo, Axios / BE - Spring Boot, Spring Security, MySQL, SSE, FCM, JWT, Postman

2. **에코 클래스룸**
   - **서비스 설명**: 수업 중 실시간으로 익명으로 의견이나 질문을 제출할 수 있는 서비스. 교수는 학생들의 이해도를 파악하고 수업 조절 가능.
   - **기술 스택**: FE - Flutter, Socket.IO, Expo CLI, Axios, TanStack Query / BE - Spring Boot, Spring Security, JWT, MySQL, Spring WebSocket, AWS

---

**입선작**
1. **Crewing**
   - **서비스 설명**: 연합 동아리 정보 플랫폼. 회원가입 시 개인 정보와 관심 분야를 입력하면 적합한 동아리 추천.
   - **기술 스택**: FE - React, Tesseract.js, React-Quill, HTML, CSS, JavaScript / BE - Java,

# 관련없는 질문 테스트
문서와 관련 없는 질문을 했을 때 모델이 문서에 기반한 답변만 제공하는지 테스트합니다.

In [50]:
unrelated_msg = "홍차에 대해 설명해줘."
user_prompt2 = prompt.invoke({"context": format_docs(retrieved_docs), "question": unrelated_msg})

response = llm.invoke(user_prompt2)
print(response.content)

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


제공된 문서에서 홍차에 대한 정보는 포함되어 있지 않습니다. 

제공된 문서에서 찾을 수 있는 관련 정보는 아래와 같습니다:

### 공모전 개요
- **행사명**: All-in 코딩 공모전
- **목적**: 대학생들이 겪는 캠퍼스의 불편함과 문제를 자신만의 아이디어로 해결하기 위한 대회
- **참여자**: 대학생들

### 공모전 결과
- **대상 수상작**: 
  - **Lexi Note**: 언어공부 필기 웹 서비스
    - 서비스 제작자: 다나와(김다애, 박나경)

- **입선 및 우수상 수상작**:
  - **Crewing**: 연합동아리 정보 플랫폼
  - **우리집 히어로즈**: 벌레 퇴치 영웅 매칭 서비스
  - **학교생활 매니저**: 학교생활 관리 서비스
  - **에코 클래스룸**: 수업 실시간 소통 서비스
  - **BLOTIE**: 교내 외국인X내국인 매칭 및 교류 플랫폼

### 프로그램 내용
- **코딩 프로그램**: 스파르타코딩클럽에서 제공하는 다양한 프로그램으로 비전공자 및 초보자도 웹/앱 개발자로 성장할 수 있도록 지원
- **기술 스택 사용 예시**:
  - 프론트엔드: React, Flutter 등
  - 백엔드: Spring Boot, MySQL 등

### 참가 독려
- 다양한 서비스와 기발한 아이디어를 가진 참가자들을 통해 캠퍼스 내 문제 해결 방법을 자랑하고 공유할 수 있는 기회 제공

위와 같은 내용이 제공된 문서에서 확인할 수 있는 정보입니다. 홍차에 대한 정보는 포함되어 있지 않으므로, 해당 정보를 제공된 문서에서 찾을 수 없습니다.


# 문제점
리트리버에서 관련된 정보를 전부 가져오지 않아 답변에서도 수상작 한 개만 가져오는 문제가 있었습니다.

## 리트리버 수정
리트리버가 반환하는 문서의 최대 개수를 지정하여 문서를 여러개 가져올 수 있도록 했습니다.
### 1. similarity 리트리버

In [ ]:
retriever_similarity = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 50})
retrieved_docs_similarity = retriever.invoke(user_msg)

print(retrieved_docs_similarity)

### 2. mmr 리트리버
MMR은 관련성과 다양성을 모두 고려하여 검색 결과의 중복을 줄이고 다양한 정보를 제공하는 데 유용합니다.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 50})
retrieved_docs_mmr = retriever.invoke(user_msg)

print(retrieved_docs_mmr)

### 3. 문서압축 리트리버
LLM을 이용해 검색된 문서를 압축하여 질문과 관련된 핵심 내용만 추출합니다.

In [ ]:
base_retriever_context = vectorstore.as_retriever(
                                search_type='similarity',
                                search_kwargs={'k':50})
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever_context
)

compressed_docs_context = compression_retriever.get_relevant_documents(user_msg)
print(len(compressed_docs_context))
compressed_docs_context

### 4. self-query 리트리버
이 방법은 메타데이터 필드를 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환하도록 합니다. 공모전과 관련된 메타데이터(상장 종류, 공모전 이름, 수상자 등)를 정의하여 보다 정확한 검색이 가능하도록 합니다.

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# 수상 정보 정의
metadata_field_info = [
    AttributeInfo(
        name="award_type",
        description="상장 종류 (대상, 최우수상, 우수상, 입선선 등)",
        type="string",
    ),
    AttributeInfo(
        name="competition_name", 
        description="공모전 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipients_name", 
        description="수상자들의의 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipient_work_decription", 
        description="수상작에 대한 설명", 
        type="string"
    ),
    AttributeInfo(
        name="tech_stack", 
        description="사용한 기술", 
        type="string"
    ),
]

self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore_chroma,
    document_contents="ALL-in 코딩 공모전 수상작 정보",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 50},
)

retrieved_docs_self_query = self_query_retriever.get_relevant_documents(user_msg)
print(len(retrieved_docs_self_query))
retrieved_docs_self_query


리트리버들의 결과 비교

In [59]:
print("1. search type: similarity")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_similarity), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


print("\n\n\n2. search type: mmr")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_mmr), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n3. contextual compression retriever")
user_prompt = prompt.invoke({"context": format_docs(compressed_docs_context), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n4. self-query retriever")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_self_query), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


1. search type: similarity


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## ALL-in 코딩 공모전 수상작 요약

### 1. 공모전 개요
- **목적**: 대학생들이 캠퍼스에서 겪는 불편함과 문제를 자신의 아이디어로 해결하는 대회
- **주최**: 스파르타코딩클럽

### 2. 수상작 목록
#### 🏆 대상
- **제목**: Lexi Note
  - **서비스 내용**: 언어 공부를 위한 필기 웹 서비스, 단어 드래그 및 번역 기능 제공
  - **서비스 제작자**: 다나와(김다애, 박나경)
  - **사용 기술 스택**: FE - Spring Boot, MySQL, Spring WebSocket / BE - React Native, TanStack Query, Axios

#### 🎖️ 우수상
1. **제목**: 우리집 히어로즈
   - **서비스 내용**: 벌레 퇴치 서비스를 제공하는 플랫폼, 사용자와 퇴치 히어로 매칭
   - **서비스 제작자**: 인트(배정연, 한지수)
   - **사용 기술 스택**: FE - React Native, Expo, Axios / BE - Spring Boot, Spring Security, MySQL, SSE, FCM, JWT, Postman

2. **제목**: 에코 클래스룸
   - **서비스 내용**: 실시간 수업 소통 플랫폼, 익명으로 의견 제출 가능
   - **서비스 제작자**: 정보 없음
   - **사용 기술 스택**: FE - Flutter, Socket.IO, Expo CLI, Axios, TanStack Query / BE - Spring Boot, Spring Security, JWT, MySQL, Spring WebSocket, AWS

#### 🏅 입선
1. **제목**: Crewing
   - **서비스 내용**: 연합동아리 정보 플랫폼, 적절한 동아리 추천 및 정보를 제공
   - **서비스 제작자**: 동학대학운동(김민아, 임경진, 신은혜, 고수)
   - **사용 기술 스택**: FE - React, Tesseract.js, React-Quill, HTM

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# ALL-in 코딩 공모전 수상작 요약

## 공모전 개요
- **공모전명**: ALL-in 코딩 공모전
- **목표**: 대학생들이 캠퍼스에서 겪은 불편함과 문제를 아이디어로 해결하기 위해 참가
- **제공 기관**: 스파르타코딩클럽

## 수상작 목록
### 1. 대상
- **프로젝트명**: Lexi Note
  - **서비스 설명**: 언어공부를 위한 필기 웹 서비스. 작업 중에 사전 검색 및 번역 기능을 제공하여 효율적인 학습을 지원함.
  - **제작자**: 다나와 (김다애, 박나경)
  - **기술 스택**: 
    - 프론트엔드: Spring Boot, MySQL, Spring WebSocket
    - 백엔드: React Native, TanStack Query, Axios

### 2. 우수상
- **프로젝트명**: 우리집 히어로즈
  - **서비스 설명**: 자취생을 위한 벌레 퇴치 영웅 매칭 서비스. 사용자가 요청하면 안전한 환경에서 벌레 퇴치 히어로와 매칭됨.
  - **제작자**: 인트 (배정연, 한지수)
  - **기술 스택**:
    - 프론트엔드: React Native, Expo, Axios
    - 백엔드: Spring Boot, Spring Security, MySQL

- **프로젝트명**: 에코 클래스룸
  - **서비스 설명**: 교수와 학생 간의 실시간 소통을 위한 플랫폼으로, 학생들이 질문을 익명으로 제출하고 교수는 피드백을 받을 수 있음.
  - **제작자**: 해당 정보를 제공된 문서에서 찾을 수 없습니다.
  - **기술 스택**:
    - 프론트엔드: Flutter, Socket.IO, Expo CLI, Axios
    - 백엔드: Spring Boot, Spring Security, JWT, MySQL 

### 3. 입선작
- **프로젝트명**: Crewing
  - **서비스 설명**: 연합동아리 정보 플랫폼으로, 적합한 동아리를 추천 및 관리.
  - **제작자**: 동학대학운동 (김민아, 

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## ALL-in 코딩 공모전 수상작 요약

### 대상
- **제목:** Lexi Note
- **설명:** 언어공부 필기 웹 서비스
- **제작자:** 다나와(김다애, 박나경)

### 우수상
- **제목:** 우리집 히어로즈
- **설명:** 벌레 퇴치 영웅 매칭 서비스
- **제작자:** 인트(배정연, 한지수)
- **사용한 기술 스택:**
  - **프론트엔드:** Flutter
  - **백엔드:** Firebase

### 입선
- **제목:** BLOTIE
- **설명:** 교내 외국인과 내국인 매칭 및 교류 플랫폼
- **제작자:** 블로티(이은주, 한명수, 황준영)

### 공모전 개요
- **목적:** 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결함. 
- **참여 특성:** 참가자들은 자신의 문제 해결 아이디어를 바탕으로 웹/앱 서비스를 개발.
  
위의 정보는 제공된 문서에서 확인할 수 있는 모든 관련 내용입니다.



4. self-query retriever


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## All-in 코딩 공모전 수상작 요약

### 1. 우수상 수상작
#### [우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스
- **제작자**: 인트(배정연, 한지수)
- **문제 배경**: 자취생 C는 벌레 문제로 불안한 밤을 보내며, 개인 정보 노출이 염려되는 커뮤니티에서 도움을 요청하기 어려워함.
- **서비스 개요**: 벌레 퇴치가 필요한 사용자가 요청을 올리면 같은 학교 학생 중 벌레 퇴치 히어로와 매칭되는 앱. 
- **주요 기능**:
  - 신원 보장된 환경에서의 매칭
  - 실시간 알림 기능
  - 상호 평가 시스템

### 2. 입선 수상작
#### [BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼
- **제작자**: 블로티(이은주, 한명수, 황준영)
- **문제 배경**: 교환학생 F는 언어 및 문화 차이로 캠퍼스 생활에 어려움을 겪음. 
- **서비스 개요**: 외국인 학생과 한국인 학생의 교류를 지원하는 플랫폼.
- **주요 기능**:
  - 1:1 매칭 기능
  - 실시간 채팅 및 피드 기능

#### [Crewing] 연합동아리 정보 플랫폼
- **제작자**: 동학대학운동(김민아, 임경진, 신은혜, 고수)
- **문제 배경**: 신입생 S는 적합한 동아리 정보를 찾기 어려운 상황.
- **서비스 개요**: 연합 동아리에 가입할 수 있도록 정보 제공.
- **주요 기능**:
  - 동아리 추천 시스템
  - 신뢰할 수 있는 후기 제공

#### [학교생활 매니저] 학교생활 관리 서비스
- **제작자**: 아이칼F4(조민제, 이민기, 강건, 박근우)
- **문제 배경**: 학교 생활의 혼란으로 주요 공지사항을 놓칠 우려가 있음.
- **서비스 개요**: 학교 생활을 효율적으로 관리하도록 돕는 앱.
- **주요 기능**:
  - 일정, 과제 관리, 성적 예측 기능
  - 실시간 공지사항 제공

#### [에코 클래스룸] 수업 실시간 소통 서비스
- **제작자**: This is 스파게티!!!(박지성, 김서원, 박범수)
- **문제

Self-Query 리트리버가 가장 효과적인 결과를 보여주었습니다. 이 방식은 메타데이터 필드(상장 종류, 공모전 이름, 수상자 등)를 명확히 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환할 수 있게 했습니다. 그 결과, 다른 리트리버들이 놓친 "BLOTIE" 입선작까지 포함한 더 포괄적인 정보를 제공했고, 프론트엔드/백엔드 기술 스택 구분, 서비스 기능 설명, 문제 해결 방식 등 더 체계적인 정보를 추출할 수 있었습니다. 특히 특정 속성에 대한 정보를 정확하게 추출해야 하는 상황에서 Self-Query 리트리버의 강점이 두드러졌으며, 이는 복잡한 정보를 구조화하여 검색하는 데 효과적인 접근법임을 보여줍니다.

## 수정사항
### vectorDB FAISS 로 변경, COSTAR 구분자 이용하여 프롬프트 작성, UnstructuredURLLoader로 변경
COSTAR로 프롬프트를 좀 더 자세하게 작성하니 가져오는 데이터가 더 구체적이었고, 각 수상작 별로 잘 정리된 체계적인 답변을 받았습니다.

contextual compression retriever는 LLM 으로 요약된 문서에서 가져오다보니 빠진 정보도 있었습니다.


self query retriever는 FAISS를 지원하지 않아 Chroma로 진행했습니다.

similarity, mmr, contextual compression retriever를 FAISS로 변경했을 때,
Chroma를 사용했을 때보다 가져오는 데이터가 정확했고, 빠트리는 데이터가 없었습니다.
